In [ ]:
using Knet: Knet, dir, accuracy, progress, sgd, load, save, gc, Param, KnetArray, Data, minibatch, nll, relu, training, dropout
using Statistics
using CUDA
using Random
using LinearAlgebra
using Knet

In [ ]:
struct Chain
    layers;
    Chain(layers...) = new(layers);
end

(c::Chain)(x) = (for l in c.layers; x = l(x); end; x);
(c::Chain)(x,y) = loss(c(x),y);
(c::Chain)(x::Tuple) = loss(c(x[1]),x[2]);
(c::Chain)(d::Data) = mean(c(x,y) for (x,y) in d);

struct Dense
    w;
    b;
    f;
end

Dense(i::Int,o::Int,f=relu) = Dense(param(o,i),param0(o),f);
(l::Dense)(x) = (l.w * x .+ l.b);




In [ ]:
function loss(Q_value,cost)
      return mean((Q_value-cost).^2)/2;
end
function sgdupdate!(func, x,y; lr=0.1)
    fval = @diff func(x,y);
    for param in params(fval)
        ∇param = grad(fval, param);
        param .-= lr * ∇param;
    end
    return value(fval);
end


In [ ]:
function get_saved_model(model_file)
    Q=[]
    push!(Q,Knet.load(model_file,"q_network1"));
    push!(Q,Knet.load(model_file,"q_network2"));
    push!(Q,Knet.load(model_file,"q_network3"));
    push!(Q,Knet.load(model_file,"q_network4"));
    push!(Q,Knet.load(model_file,"q_network,5"));
    push!(Q,Knet.load(model_file,"q_network6"));
    
    return Q
end

In [ ]:
struct database
    turn_depth;
    data;
end

function database(t=6)
    data=[]
    for i in 1:t
        push!(data,[])
    end
    return database(t,data)
end

function last_n_data(data, n_element=50000)
    for i in 1:length(data)
        if length(data[i]) > n_element
            data[i]= data[length(data[i])-n_element:end]
        end
    end
end

function Base.push!(d::database,t,element)
    push!(d.data[t],element)
end